In [1]:
#importing libraries

from bs4 import BeautifulSoup as Soup
import urllib.request
import unicodedata
import json
import csv
# end 

In [2]:
# function for parsing data

def parse(url = '',class_string = ''):
    request=urllib.request.Request(url,None,headers)
    response = urllib.request.urlopen(request)
    data = response.read()
    parsed_data = Soup(data,'lxml')
    division = parsed_data.find('div',class_ = class_string)
    return division
# end

In [10]:
# function for creating json file

def create_json(question_list,answer_list,topics):
    ques_ans_list = []
    for i in range(len(question_list)):
        temp = {question_list[i][j]:answer_list[i][j] for j in range(len(question_list[i]))}
        ques_ans_list.append(temp)
    topic_dict = {topics[i] : ques_ans_list[i] for i in range(len(topics))}
    with open('data/topic_list.json','w') as f:
        json.dump(topic_dict,f)
# end

In [16]:
# function for creating csv file from question answers 

def create_csv(question_list,answer_list):
    question_flat = [question for  questions in question_list for question in questions]
    answer_flat = [answer for answers in answer_list for answer in answers]
    with open('data/ques_ans.csv','w',encoding = 'utf-8') as file:
        csvwriter = csv.writer(file)
        csvwriter.writerow(['Context','Answer'])
        for i in range(len(question_flat)):
            csvwriter.writerow([question_flat[i],answer_flat[i]])
# end

In [6]:
#initializing common content

user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,}
# end

In [7]:
# accessing main page and getting links and topics

url = 'https://www.who.int/emergencies/diseases/novel-coronavirus-2019/question-and-answers-hub'
cstring = 'sf-list-vertical sf-list-vertical__nodate'
parsed_data = parse(url,cstring)
links = []
topics = []
for data in parsed_data.find_all('a'):
    topics.append(data.span.text)
    links.append('https://www.who.int/'+data['href'])
# end

In [8]:
# scrapping questions and answers and putting them in the list

cstring = 'sf-accordion'
question_list = []
answer_list = []
for link in links:
    parsed_data = parse(link,cstring)
    questions = []
    answers = []
    for data in parsed_data.find_all('div',class_ = "sf-accordion__panel"):
        questions.append(data.a.text)
        answers.append(data.find('div',class_ = 'sf-accordion__content').text)
    questions = [question.strip() for question in questions]
    answers = [answer.strip() for answer in answers]
    answers = [unicodedata.normalize('NFKD',answer) for answer in answers]
    question_list.append(questions)
    answer_list.append(answers)
# end  

In [18]:
# sending data to files

create_json(question_list,answer_list,topics)
create_csv(question_list,answer_list)
#end